# Многопоточность и Многопроцессорность

Помните поговорку "много рук делают работу легче". Это верно в программировании так же, как и в любых других делах.

Что, если Вы так напишете программу Python, чтобы она делала четыре вещи одновременно? Тогда та работа, которая выполняется за один час, могла бы выполниться (почти) в четыре раза быстрее.<font color=green>\*</font>

Это основная идея параллельных вычислений - возможность настроить и выполнять несколько задач одновременно.


<br><font color=green>\* *Мы говорим "почти", потому что ещё потребуется время на настройку четырёх процессоров, а также время на сбор информации от них.*</font>

##  Потоки (Threading) vs. Процессы (Processing)

Хорошая иллюстрация для потоков и процессов - это загрузка файла с изображением, и создание для него иконки.

Первая часть, взаимодействие с внешним источником для загрузки файла, это отдельный поток. Как только файл получен, работа по его конвертации это процесс. По сути, время выполнения определяется двумя факторами: скорость ввода/вывода по сети, или I/O, и доступный процессор, CPU.

#### I/O-интенсивные задачи ускоряются с помощью многопоточности:
* чтение множества web-страниц
* чтение и запись файлов
* обмен данными между программами
* сетевые взаимодействия


#### CPU-интенсивные задачи ускоряются с помощью многопроцессорности:
* вычисления
* форматирование текста
* изменение масштаба изображений
* анализ данных

## Пример на многопоточность: чтение множества web-страниц (Webscraping)

Если взглянуть на историю, то навыки для настройки многопоточности были за пределами рамок этого курса, поскольку раньше требовалось хорошее знание Python's Global Interpreter Lock (GIL делает так, чтобы разные потоки не выполняли одновременно один и тот же код). Также требовалось создавать специальные классы, которые выполняют роль Поставщиков заданий (Producers) для разделения работы на части, классы для Потребителей заданий (Consumers, или "workers") для выполнения работы, и классов Queue (очередь) для хранения задач и обеспечения взаимодействия. И это было только начало.

К счастью, мы уже изучили один полезный инструмент, который нам понадобится – функция `map()`. Когда мы применяем её, используя две стандартные библиотеки, *multiprocessing* и *multiprocessing.dummy*, то настройка параллельных процессов и потоков выполняется достаточно просто.


Ниже приведён классический пример многопоточности, предоставленный [IBM](http://www.ibm.com/developerworks/aix/library/au-threadingpython/) и адаптированный [Chris Kiehl](http://chriskiehl.com/article/parallelism-in-one-line/), в котором Вы разделяете задачу чтения web-страниц между несколькими потоками:


    import time 
    import threading 
    import Queue 
    import urllib2 

    class Consumer(threading.Thread): 
      def __init__(self, queue): 
        threading.Thread.__init__(self)
        self._queue = queue 

      def run(self):
        while True: 
          content = self._queue.get() 
          if isinstance(content, str) and content == 'quit':
            break
          response = urllib2.urlopen(content)
        print 'Thanks!'


    def Producer():
      urls = [
        'http://www.python.org', 'http://www.yahoo.com'
        'http://www.scala.org', 'http://www.google.com'
        # etc.. 
      ]
      queue = Queue.Queue()
      worker_threads = build_worker_pool(queue, 4)
      start_time = time.time()

      # Add the urls to process
      for url in urls: 
        queue.put(url)  
      # Add the poison pill
      for worker in worker_threads:
        queue.put('quit')
      for worker in worker_threads:
        worker.join()

      print 'Done! Time taken: {}'.format(time.time() - start_time)

    def build_worker_pool(queue, size):
      workers = []
      for _ in range(size):
        worker = Consumer(queue)
        worker.start() 
        workers.append(worker)
      return workers

    if __name__ == '__main__':
      Producer()

С помощью многопоточной библиотеки, предоставляемой модулем *multiprocessing.dummy* и функцией `map()`, все это превращается в следующий код:

    import urllib2
    from multiprocessing.dummy import Pool as ThreadPool
    
    pool = ThreadPool(4) # choose a number of workers
    
    urls = [
    'http://www.python.org', 'http://www.yahoo.com'
    'http://www.scala.org', 'http://www.google.com'
    # etc.. 
    ]
    
    results = pool.map(urllib2.urlopen, urls)
    pool.close() 
    pool.join()
    
В примере выше, модуль *multiprocessing.dummy* предоставляет параллельные потоки (threads), а функция `map(urllib2.urlopen, urls)` назначает работу!

## Пример на мультипроцессорность: Монте Карло

Давайте напишем код для примера, чтобы увидеть, как различные части сочетаются между собой. Мы можем измерить время с помощью модуля *timeit*, чтобы понять выигрыш по производительности. Задача будет в том, чтобы применить Метод Монте Карло для оценки значения числа Pi.

### Метод Монте Карло и оценка числа Pi

Если Вы нарисуете окружность радиуса 1 и заключите её в квадрат, то площадь этих двух фигур будет равна

<table>
    <caption>Area Formulas</caption>
    <tr><td>circle</td><td>$$πr^2$$</td></tr>
    <tr><td>square</td><td>$$4 r^2$$</td></tr>
</table>


Следовательно, отношение площади круга к площади квадрата равно $$\frac{π}{4}$$

Метод Монте Карло генерирует набор случайных точек внутри квадрата. Далее сравниваем количество точек, которые попали внутрь круга, и тех, которые не попали в него. И при большом количестве точек мы можем получить хорошую оценку для числа Pi. Вы можете посмотреть хорошую демонстрацию [здесь](https://academo.org/demos/estimating-pi-monte-carlo/) (Нажмите кнопку **Animate** на той странице).

Для заданного количества точек *n*, у нас будет $$π = \frac{4 \cdot points\ inside\ circle}{total\ points\ n}$$

Чтобы создать многопроцессорную программу, сначала мы создаём функцию для поиска Pi, которую мы передадим в `map()`:

In [1]:
from random import random  # perform this import outside the function

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

Теперь проверим функцию `find_pi` для 5,000 точек:

In [2]:
find_pi(5000)

3.1064

Выполнилось очень быстро, но результаты не очень точные!

Далее мы напишем скрипт для создания набора процессов, и измерим время выполнения для различного количества этих процессов. Также у нас будет два параметра - *processes* и *total_iterations*. Внутри скрипта мы разобъём *total_iterations* на отдельные части, которые передадим каждому из процессов, создав список для каждого из процессов.<br>Например:

    total_iterations = 1000
    processes = 5
    iterations = [total_iterations//processes]*processes
    iterations
    # Output: [200, 200, 200, 200, 200]
    
Этот список мы передадим в функцию `map()` вместе с `find_pi()`

In [3]:
%%writefile test.py
from random import random
from multiprocessing import Pool
import timeit

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    N = 10**5  # total iterations
    P = 5      # number of processes
    
    p = Pool(P)
    print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total iterations with {P} processes')

Writing test.py


In [4]:
! python test.py

3.1466800
3.1364400
3.1470400
3.1370400
3.1256400
3.1398400
3.1395200
3.1363600
3.1437200
3.1334400
0.2370227286270967
100000 total iterations with 5 processes


Отлично! Этот тест занял меньше секунды на нашем компьютере.

Теперь, когда мы знаем, что скрипт работает, давайте увеличим количество итераций, и сравним два различных пула процессов. Можете откинуться на списку кресла, это займёт некоторое время!

In [5]:
%%writefile test.py
from random import random
from multiprocessing import Pool
import timeit

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    N = 10**7  # total iterations
    
    P = 1      # number of processes
    p = Pool(P)
    print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total iterations with {P} processes')
    
    P = 5      # number of processes
    p = Pool(P)
    print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total iterations with {P} processes')

Overwriting test.py


In [6]:
! python test.py

3.1420964
3.1417412
3.1411108
3.1408184
3.1414204
3.1417656
3.1408324
3.1418828
3.1420492
3.1412804
36.03526345242264
10000000 total iterations with 1 processes
3.1424524
3.1418376
3.1415292
3.1410344
3.1422376
3.1418736
3.1420540
3.1411452
3.1421652
3.1410672
17.300921846344366
10000000 total iterations with 5 processes



Надеюсь Вы увидели, что с 5 процессами наш скрипт выполняется быстрее!

## Больше это лучше... до определённого момента.

По мере увеличения процессов, выигрыш по производительности с какого-то момента становится всё меньше и меньше, и выходит на плато. Дело в том, что в наборе задач будет одна или две задачи, которые занимают больше, чем среднее значение, и они не будут ускорены добавлением дополнительных ресурсов. Это лучше всего описано в [Законе Амдала](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%90%D0%BC%D0%B4%D0%B0%D0%BB%D0%B0).

## Продвинутый скрипт

В примере ниже мы добавляем контекстный менеджер, чтобы сжать эти три строки

    p = Pool(P)
    ...
    p.close()
    p.join()
    
в одну строку:

    with Pool(P) as p:
    
И мы будем принимать параметры командной строки, используя модуль *sys*.
    

In [7]:
%%writefile test2.py
from random import random
from multiprocessing import Pool
import timeit
import sys

N = int(sys.argv[1])  # these arguments are passed in from the command line
P = int(sys.argv[2])

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    
    with Pool(P) as p:
        print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.5f}'), number=10))
    print(f'{N} total iterations with {P} processes')

Writing test2.py


In [8]:
! python test2.py 10000000 500

3.14121
3.14145
3.14178
3.14194
3.14109
3.14201
3.14243
3.14150
3.14203
3.14116
16.871822701405073
10000000 total iterations with 500 processes


Отлично! Теперь у Вас должно сложиться понимание многопоточности и многопроцессорности!